In [1]:
%load_ext autoreload
%autoreload 2

import dataclasses

import torch
from torch import nn

In [2]:
from src.networks.seq_net import SeqNet
from src.networks.torch_net import TorchNet
net = SeqNet([TorchNet(nn.Sequential(
    nn.ReLU(),
    nn.ReLU(),
))])
print(net.in_features, net.out_features)
net


any same


SeqNet(
  (layers): NetList(
    (0): TorchNet(
      (torch_module): Sequential(
        (0): ReLU()
        (1): ReLU()
      )
    )
  )
)

In [41]:
from src.networks.dense_layer_stack import AdditiveDenseLayerStack, ConcatDenseLayerStack
dense_layer_stack = AdditiveDenseLayerStack(
    lambda _1, _2, _3, _4: nn.Identity(), 
    num_layers=3, 
    num_features=8,
    initial_direct_connection_weight = 3,
    initial_skip_connection_weight = 2,
    connections = 'dense',
    weights_trainable=True,
)
count_parameters(dense_layer_stack)
# dense_layer_stack = ConcatDenseLayerStack(
#     lambda i, is_final_layer, in_size, out_size: nn.Sequential(
#         nn.Linear(in_size, out_size)
#     ),
#     layers_sizes=[5, 3, 7, 2],
#     connections='dense'
# )
# print(dense_layer_stack)
# dense_layer_stack(torch.Tensor([11] * 5), return_dense=True)

128

In [2]:
from src.networks.forward_net import ForwardNet
ForwardNet(
    layer_provider=lambda i, last, in_f, out_f: nn.Sequential(nn.Linear(in_f, out_f), nn.ReLU()),
    layers_in_out_sizes: list[tuple[int, int]] = None,
    layers_sizes: list[int] = None,
    num_layers: int = None,
    num_features: int = None,
    
)

SyntaxError: positional argument follows keyword argument (2220535212.py, line 4)

In [12]:
from src.networks.forward_net import ForwardNet
ForwardNet(layer_provider=lambda _1, _2, _3, _4: nn.Identity(), in_size=1, out_sizes=[2, 3])

ForwardNet(
  (layers): Sequential(
    (0): Identity()
    (1): Identity()
  )
)

In [14]:
from src.networks.fnn import FNN
FNN.self_normalizing(in_size=1, out_sizes=[2, 3])

FNN(
  (layers): Sequential(
    (0): Sequential(
      (0): Linear(in_features=1, out_features=2, bias=True)
      (1): SELU()
    )
    (1): Sequential(
      (0): Linear(in_features=2, out_features=3, bias=True)
    )
  )
)

In [45]:
np.cumsum([1, 2, 3])

array([1, 3, 6])

In [19]:
FNN.self_normalizing(
    1, 
    [2, 5], 
    3, 
    activate_last_layer=True, 
    # dropout_p=0.1, 
    # dropout_last_layer=True
)

FNN(
  (fnn): Sequential(
    (0): Linear(in_features=1, out_features=2, bias=True)
    (1): SELU()
    (2): Linear(in_features=2, out_features=5, bias=True)
    (3): SELU()
    (4): Linear(in_features=5, out_features=3, bias=True)
    (5): SELU()
  )
)

In [35]:
transformer = TransformerEncoder(
    num_layers=8,
    num_features=32,
    attention_num_heads=4,
    feedforward_provider=lambda num_features: FNN(
        input_size=num_features, hidden_sizes=[256], output_size=num_features
    ),
    residual_connection_weight_affine=True,
)
print(count_parameters(transformer))

# nn_transformer = nn.TransformerEncoder(
#     nn.TransformerEncoderLayer(
#         d_model=512,
#         nhead=8,
#     ),
#     num_layers=6
# )
# print(count_parameters(nn_transformer))

# count_parameters(transformer) / count_parameters(nn_transformer)

168704


18914304

1.0